In [8]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-

In [9]:
import argparse
import gzip
import math
import numpy
import re
import sys
from nltk.corpus import wordnet as wn
from copy import deepcopy

In [2]:
# TODO: keep?

isNumber = re.compile(r'\d+.*')

def norm_word(word):
  """
  - input: word
  - return: a normalized version of it
  Normalization process: includes checking if the word is a number or a punctuation mark and replacing it with special tokens
  """
  if isNumber.search(word.lower()):
    return '---num---'
  # check if the word consists only of non-alphanumeric characters by removing all non-alphanumeric characters from the word 
  # and checking if the result is an empty string
  elif re.sub(r'\W+', '', word) == '':
    return '---punc---'
  else:
  # if input word not a number nor a punctuation mark, return a lowercase version of input word
    return word.lower()

In [3]:
''' Read all the word vectors and normalize them '''
def read_word_vecs(filename):
  """
  - input: name of the file containing the word vectors
  """
  wordVectors = {}
  with open(filename, 'r', encoding='utf-8') as fileObject:
    for line in fileObject:
      line = line.strip().lower()
      # The first word is assumed to be the word itself, and the remaining words are assumed to be the components of the word vector
      word = line.split()[0]
      # initialize a numpy array of zeros with the same length as the word vector
      wordVectors[word] = numpy.zeros(len(line.split())-1, dtype=float)
      for index, vecVal in enumerate(line.split()[1:]):
        # assign the values in the numpy array to the corresponding components of the word vector
        wordVectors[word][index] = float(vecVal)
      ''' normalize weight vector '''
      # divide each element by the square root of the sum of the squares of all the elements in the array
      # plus a small constant (1e-6) to avoid division by zero
      wordVectors[word] /= math.sqrt((wordVectors[word]**2).sum() + 1e-6)
  
  # standard error indicating that the vectors have been read from the file 
  sys.stderr.write("Vectors read from: "+filename+" \n")
  return wordVectors

In [4]:
''' Write word vectors to file '''
def print_word_vecs(wordVectors, outFileName):
  """
  - input: a dictionary wordVectors where keys are words and values are their corresponding word vectors
           file name outFileName
  """
  sys.stderr.write('\nWriting down the vectors in '+outFileName+'\n')
  outFile = open(outFileName, 'w', encoding= 'utf-8')  
  for word, values in wordVectors.items():
    outFile.write(word+' ')
    for val in wordVectors[word]:
      # write the word vectors to the ouptu file in the format:
      # word1 val1 val2 val3 ...
      # word2 val1 val2 val3 ...
      # ...
      outFile.write('%.4f' %(val)+' ')
    outFile.write('\n')      
  outFile.close()

In [5]:
''' Read the PPDB word relations as a dictionary '''
def read_lexicon(filename):
  lexicon = {}
  for line in open(filename, 'r'):
    words = line.lower().strip().split()
    lexicon[norm_word(words[0])] = [norm_word(word) for word in words[1:]]
  return lexicon

In [10]:
# Hyperparameters
ALPHA = 1 #coefficient minimizing the euclidean distance
BETA = 1/ len(lexicon) # minimizes each vector distance to ?
NB_ITER = 10

''' Retrofit word vectors to a lexicon '''
def retrofit(wordVecs, lexicon):
  newWordVecs = deepcopy(wordVecs)
  wvVocab = set(newWordVecs.keys())
  loopVocab = wvVocab.intersection(set(lexicon.keys()))
  for _ in range(NB_ITER):
    # loop through every node also in ontology (else just use data estimate)
    for word in loopVocab:
      wordNeighbours = set(lexicon[word]).intersection(wvVocab)
      numNeighbours = len(wordNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue
      # the weight of the data estimate if the number of neighbours
      newVec = ALPHA * wordVecs[word] + BETA * sum(newWordVecs[ppWord] for ppWord in wordNeighbours) 
      newWordVecs[word] = newVec/ (ALPHA + BETA * numNeighbours)
  return newWordVecs


In [ ]:
toy_corpus = ["cat", "dog", "caramel", "cheese", "chocolate", "right", "left"]


In [11]:
input_file = "../data/English/wordEmbeddings/vectors_datatxt_250_sg_w10_i5_c500_gensim_clean"
lexicon_file = "../data/English/lexicon/ws353_lexical_similarity.txt"
output_file = "../data/English/output_vectors/output_vectors.txt"

wordVecs = read_word_vecs(input_file)
lexicon = read_lexicon(lexicon_file)
outFileName = output_file

# ''' Enrich the word vectors using ppdb and print the enriched vectors '''
# print_word_vecs(retrofit(wordVecs, lexicon, numIter), outFileName)
retrofittedVecs = retrofit(wordVecs, lexicon)

with open(output_file, 'w', encoding='utf-8') as outputFile:
    for word in retrofittedVecs.keys():
        outputFile.write(word + ' ' + ' '.join(str(x) for x in retrofittedVecs[word]) + '\n')


Vectors read from: ../data/English/wordEmbeddings/vectors_datatxt_250_sg_w10_i5_c500_gensim_clean 
